In [79]:
import sys
import smaract.ctl as ctl

In [118]:
def assert_lib_compatibility():
    """
    Checks that the major version numbers of the Python API and the
    loaded shared library are the same to avoid errors due to 
    incompatibilities.
    Raises a RuntimeError if the major version numbers are different.
    """
    vapi = ctl.api_version
    vlib = [int(i) for i in ctl.GetFullVersionString().split('.')]
    if vapi[0] != vlib[0]:
        raise RuntimeError("Incompatible SmarActCTL python api and library version.")

In [119]:
version = ctl.GetFullVersionString()
print("SmarActCTL library version: '{}'.".format(version))
assert_lib_compatibility()

SmarActCTL library version: '1.4.3.134888'.


In [120]:
locator = "usb:ix:0"


In [128]:
d_handle = None
# try:
d_handle = ctl.Open(locator)
print("MCS2 opened {}.".format(locator))


MCS2 opened usb:ix:0.


In [122]:
device_sn = ctl.GetProperty_s(d_handle, 0, ctl.Property.DEVICE_SERIAL_NUMBER)
print("MCS2 device serial number: {}".format(device_sn))

MCS2 device serial number: MCS2-00002553


In [123]:
no_of_channels = ctl.GetProperty_i32(d_handle, 0, ctl.Property.NUMBER_OF_CHANNELS)
print("MCS2 number of channels: {}.".format(no_of_channels))

MCS2 number of channels: 6.


In [124]:
for channel in range(no_of_channels):
    state = ctl.GetProperty_i32(d_handle, channel, ctl.Property.CHANNEL_STATE)
    # The returned channel state holds a bit field of several state flags.
    # See the MCS2 Programmers Guide for the meaning of all state flags.
    # We pick the "sensorPresent" flag to check if there is a positioner connected
    # which has an integrated sensor.
    # Note that in contrast to previous controller systems the controller supports
    # hotplugging of the sensor module and the actuators.
    if state & ctl.ChannelState.SENSOR_PRESENT:
        print("MCS2 channel {} has a sensor.".format(channel))
    else:
        print("MCS2 channel {} has no sensor.".format(channel))

MCS2 channel 0 has a sensor.
MCS2 channel 1 has a sensor.
MCS2 channel 2 has a sensor.
MCS2 channel 3 has a sensor.
MCS2 channel 4 has a sensor.
MCS2 channel 5 has a sensor.


In [129]:
  # For the following steps we need a positioner connected to channel 0.
channel = 0

    # First we want to know if the configured positioner type is a linear or a rotatory type.
    # For this purpose we can read the base unit property.
base_unit = ctl.GetProperty_i32(d_handle, channel, ctl.Property.POS_BASE_UNIT)

In [126]:
position = ctl.GetProperty_i64(d_handle, channel, ctl.Property.POSITION)
print("MCS2 position of channel {}: {}".format(channel, position), end='')
print("pm.") if base_unit == ctl.BaseUnit.METER else print("ndeg.")

MCS2 position of channel 0: -141982731pm.


In [57]:
position = 100000000 # in pm | ndeg
print("MCS2 set position of channel {} to {}".format(channel, position), end='')
print("pm.") if base_unit == ctl.BaseUnit.METER else print("ndeg.")
ctl.SetProperty_i64(d_handle, channel, ctl.Property.POSITION, position)

MCS2 set position of channel 0 to 100000000pm.


In [58]:
# Issue requests for the two properties "position" and "channel state".
r_id1 = ctl.RequestReadProperty(d_handle, channel, ctl.Property.POSITION, 0)
# The function call returns immediately, allowing the application to issue another request or to perform other tasks.
# We simply request a second property. (the channel state in this case)
r_id2 = ctl.RequestReadProperty(d_handle, channel, ctl.Property.CHANNEL_STATE, 0)

In [59]:
position = ctl.ReadProperty_i64(d_handle, r_id1)
state = ctl.ReadProperty_i32(d_handle, r_id2)

    # Print the results
print("MCS2 current position of channel {}: {}".format(channel, position), end='')
print("pm.") if base_unit == ctl.BaseUnit.METER else print("ndeg.")
if (state & ctl.ChannelState.ACTIVELY_MOVING) == 0:
    print("MCS2 channel {} is stopped.".format(channel))

MCS2 current position of channel 0: 100029398pm.
MCS2 channel 0 is stopped.


In [60]:
position = -100000000
print("MCS2 set position of channel {} to {}".format(channel, position), end='')
print("pm.") if base_unit == ctl.BaseUnit.METER else print("ndeg.")
r_id = ctl.RequestWriteProperty_i64(d_handle, channel, ctl.Property.POSITION, position)
# The function call returns immediately, without waiting for the reply from the controller.
# ...process other tasks...

# Wait for the result to arrive.
ctl.WaitForWrite(d_handle, r_id)

MCS2 set position of channel 0 to -100000000pm.


In [61]:
ctl.RequestWriteProperty_i64(d_handle, channel, ctl.Property.POSITION, position, pass_rID = False)


In [127]:
if d_handle != None:
    ctl.Close(d_handle)

In [63]:
def calibrate(channel):
    print("MCS2 start calibration on channel: {}.".format(channel))
    # Set calibration options (start direction: forward)
    ctl.SetProperty_i32(d_handle, channel, ctl.Property.CALIBRATION_OPTIONS, 0)
    # Start calibration sequence
    ctl.Calibrate(d_handle, channel)
    # Note that the function call returns immediately, without waiting for the movement to complete.
    # The "ChannelState.CALIBRATING" flag in the channel state can be monitored to determine
    # the end of the calibration sequence.


In [71]:
buffer = ctl.FindDevices()
if len(buffer) == 0:
    print("MCS2 no devices found.")
    raise ConnectionError
locators = buffer.split("\n")
for locator in locators:
    print("MCS2 available devices: {}".format(locator))
    

MCS2 no devices found.


ConnectionError: 

In [140]:
d_handle=None
d_handle = ctl.Open(locators)
# print("MCS2 opened {}.".format(locators[0]))
channel = 0


AttributeError: 'list' object has no attribute 'encode'

In [130]:
def calibrate(channel):
    print("MCS2 start calibration on channel: {}.".format(channel))
    # Set calibration options (start direction: forward)
    ctl.SetProperty_i32(d_handle, channel, ctl.Property.CALIBRATION_OPTIONS, 0)
    # Start calibration sequence
    ctl.Calibrate(d_handle, channel)

In [131]:
calibrate(channel)

MCS2 start calibration on channel: 0.


In [116]:
def move(channel, move_mode, direction):
    # Set move mode depending properties for the next movement.
    if move_mode == ctl.MoveMode.CL_ABSOLUTE:
        # Set move velocity [in pm/s].
        ctl.SetProperty_i64(d_handle, channel, ctl.Property.MOVE_VELOCITY, 1000000000)
        # Set move acceleration [in pm/s2].
        ctl.SetProperty_i64(d_handle, channel, ctl.Property.MOVE_ACCELERATION, 1000000000)
        # Specify absolute position [in pm].
        # (For Piezo Scanner channels adjust to valid value within move range, e.g. +-10000000.)
        move_value = 1000000000
        if direction:
            move_value = -2000000000
        print("MCS2 move channel {} to absolute position: {} pm.".format(channel, move_value))
    elif move_mode == ctl.MoveMode.CL_RELATIVE:
        # Set move velocity [in pm/s].
        ctl.SetProperty_i64(d_handle, channel, ctl.Property.MOVE_VELOCITY, 500000000)
        # Set move acceleration [in pm/s2].
        ctl.SetProperty_i64(d_handle, channel, ctl.Property.MOVE_ACCELERATION, 10000000000)
        # Specify relative position distance [in pm] and direction.
        # (For Piezo Scanner channels adjust to valid value within move range, e.g. 10000000.)
        move_value = 500000000
        if direction:
            move_value = -move_value
        print("MCS2 move channel {} relative: {} pm.".format(channel, move_value))
    elif move_mode == ctl.MoveMode.SCAN_ABSOLUTE:
        # Set scan velocity [in dac increments/s].
        # Valid range (Stick-Slip Piezo): 1 to 65535000000
        # Valid range (Piezo Scanner): 1 to 4194300000
        ctl.SetProperty_i64(d_handle, channel, ctl.Property.SCAN_VELOCITY, (65535*2))
        # Specify absolute scan target to which to scan to [in dac increments].
        # Valid range (Stick-Slip Piezo): 0 to 65535 corresponding to 0 to 100V piezo voltage
        # Valid range (Piezo Scanner): 0 to 1048575 corresponding to -20 to 100V piezo voltage
        move_value = 65535
        if direction:
            move_value = 0
        print("MCS2 scan channel {} absolute to: {}.".format(channel, move_value))
    elif move_mode == ctl.MoveMode.SCAN_RELATIVE:
        # Set scan velocity [in dac increments/s].
        ctl.SetProperty_i64(d_handle, channel, ctl.Property.SCAN_VELOCITY, 65535)
        # Specify relative scan target and direction to which to scan to [in dac increments].
        # Valid range( Stick-Slip Piezo): -65535 to 65535 corresponding to 0 to 100V piezo voltage
        # Valid range (Piezo Scanner): -1048575 to 1048575 corresponding to -20 to 100V piezo voltage
        # If the resulting absolute scan target exceeds the valid range the scan movement will stop at the boundary.
        move_value = 65535
        if direction:
            move_value = -move_value
        print("MCS2 scan channel {} relative: {}.".format(channel, move_value))
    elif move_mode == ctl.MoveMode.STEP: # (only Stick-Slip piezo positioners)
        # Set step frequency [in Hz].
        # Valid range: 1 to 20000 Hz
        ctl.SetProperty_i32(d_handle, channel, ctl.Property.STEP_FREQUENCY, 1000)
        # Set maximum step amplitude [in dac increments].
        # valid range: 0 to 65535 corresponding to 0 to 100V piezo voltage
        # Lower amplitude values result in smaller step width.
        ctl.SetProperty_i32(d_handle, channel, ctl.Property.STEP_AMPLITUDE, 65535)
        # Specify the number of steps to perform and the direction.
        move_value = 500
        if direction:
            move_value = -move_value
        print("MCS2 open loop step move, channel {}, steps: {}.".format(channel, move_value))
    # Start actual movement.
    ctl.Move(d_handle, channel, move_value, 0)
    # Note that the function call returns immediately, without waiting for the movement to complete.
    # The "ChannelState.ACTIVELY_MOVING" (and "ChannelState.CLOSED_LOOP_ACTIVE") flag in the channel state
    # can be monitored to determine the end of the movement.


In [138]:
move(channel, ctl.MoveMode.STEP, True)


Error: SetProperty_i32 returned 61447 with arguments {'dHandle': 4, 'idx': 0, 'pkey': <Property.STEP_FREQUENCY: 50659374>, 'value': 1000}

In [137]:
if d_handle != None:
        ctl.Close(d_handle)